## 기업 재무지표 적재

In [19]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from sqlalchemy import create_engine, text
import time
import sys
sys.path.append(str(Path.cwd().parent))

from dotenv import load_dotenv
load_dotenv()

DB_USER = os.getenv("POSTGRE_USER")
DB_PASSWORD = os.getenv("POSTGRE_PASSWORD")
DB_HOST = os.getenv("POSTGRE_HOST", "localhost")
DB_PORT = os.getenv("POSTGRE_PORT", "5432")
DB_NAME = os.getenv("POSTGRE_DB")

db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(db_url)
   
# 기업 재무 데이터 수집 모듈
from app.crawler.dart_loader import get_dart_report_fs

# 기업 코드
with engine.begin() as conn:
    df_corp_map = pd.read_sql("SELECT corp_code, stock_name FROM dart_corp_map", conn)
    
create_table_sql = """
CREATE TABLE IF NOT EXISTS dart_report_ofs (
    corp_code TEXT NOT NULL,
    corp_name TEXT NOT NULL,
    year INT NOT NULL,
    revenue BIGINT DEFAULT 0,           -- 매출액
    operating_profit BIGINT DEFAULT 0,  -- 영업이익
    net_profit BIGINT DEFAULT 0,        -- 당기순이익
    PRIMARY KEY (corp_code, year)
);
"""

create_table_sql2 = """
CREATE TABLE IF NOT EXISTS dart_report_cfs (
    corp_code TEXT NOT NULL,
    corp_name TEXT NOT NULL,
    year INT NOT NULL,
    revenue BIGINT DEFAULT 0,           -- 매출액
    operating_profit BIGINT DEFAULT 0,  -- 영업이익
    net_profit BIGINT DEFAULT 0,        -- 당기순이익
    PRIMARY KEY (corp_code, year)
);
"""

### 1. 옵션 설정
   - fs_names : '재무제표' or '연결재무제표'
   - indicators : account_nm 내 재무지표명
   - year : 2024 ~ 2020 진행

In [20]:
fs_names = '연결재무제표'
indicators = ['매출액','영업이익','당기순이익']
corp_code_list = list(df_corp_map.corp_code)

### 2. 재무 데이터 수집 시작

In [21]:
fs_data = pd.DataFrame()
for idx, corp_code in enumerate(corp_code_list):
    print("##########")
    print(corp_code,"::",idx+1,"/",len(corp_code_list))
    for year in [2024, 2022]:
        result = get_dart_report_fs(corp_code, year, fs_names, indicators)
        fs_data = pd.concat([fs_data, result], axis=0, ignore_index=True)
        time.sleep(2)  

##########
00266961 :: 1 / 6
##########
00258801 :: 2 / 6
##########
01547845 :: 3 / 6
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

[WARN] 재무제표 없음: 01547845(2024)
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

[WARN] 재무제표 없음: 01547845(2022)
##########
01717824 :: 4 / 6
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

[WARN] 재무제표 없음: 01717824(2024)
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

[WARN] 재무제표 없음: 01717824(2022)
##########
00126186 :: 5 / 6
##########
00139834 :: 6 / 6
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

[WARN] 재무제표 없음: 00139834(2022)


In [22]:
# 중복 제거
fs_data = fs_data.drop_duplicates()

# 컬럼명, 기업명 매핑
code_map = df_corp_map.set_index('corp_code')['stock_name'].to_dict()
indicators_map = {
    "매출액": "revenue",
    "영업이익": "operating_profit",
    "당기순이익": "net_profit"
}

fs_data = fs_data.rename(columns=indicators_map)
corp_name_col = fs_data['corp_code'].map(code_map)
fs_data.insert(1, 'corp_name', corp_name_col) 

In [24]:
# 카카오가 중복된 2022년도 매출액 값이 이상해서 임의삭제
fs_data = fs_data.drop(index=9)
fs_data = fs_data.reset_index(drop=True)

In [26]:
# 씨엔에스 중복된 2022년도 매출액 값이 이상해서 임의삭제
fs_data = fs_data.drop(index=28)
fs_data = fs_data.reset_index(drop=True)

In [27]:
fs_data

,corp_code,corp_name,year,revenue,operating_profit,net_profit
0,00266961,NAVER,2024,10737719264647,1979263438827,1931976372953
1,00266961,NAVER,2023,9670643576585,1488820269608,985017762493
2,00266961,NAVER,2022,8220078708810,1304664410543,673180300087
3,00266961,NAVER,2021,6817599707250,1325478171117,16477625559645
4,00266961,NAVER,2020,5304145900386,1215342457273,844996977149
5,00258801,카카오,2024,7871692199887,460212161525,-161870567171
6,00258801,카카오,2023,7557001757272,460857845673,-1816669011014
7,00258801,카카오,2022,6798741511168,569390445168,1067029037423
8,00258801,카카오,2021,0,587884339202,1640484195406
9,00258801,카카오,2020,0,450519574781,168978904480


### 3. 적재

#### 재무제표

In [17]:
with engine.begin() as conn:
    conn.execute(text(create_table_sql))

fs_data.to_sql(
    'dart_report_ofs',
    con=engine,
    if_exists='append',  # append: 데이터 추가
    index=False
)

30

연결재무제표

In [28]:
with engine.begin() as conn:
    conn.execute(text(create_table_sql2))

fs_data.to_sql(
    'dart_report_cfs',
    con=engine,
    if_exists='append',  # append: 데이터 추가
    index=False
)

30

### 4. 적재 확인

In [18]:
query = "SELECT * FROM dart_report_ofs;"
check = pd.read_sql(query, engine)
check

,corp_code,corp_name,year,revenue,operating_profit,net_profit
0,00266961,NAVER,2024,6180892084942,1905258322935,1968704486363
1,00266961,NAVER,2023,5609799005883,1713205368367,1414312847932
2,00266961,NAVER,2022,5512586322305,1553794848099,1092157607856
3,00266961,NAVER,2021,5018667294374,1558728780279,1524755009961
4,00266961,NAVER,2020,4126629312584,1440813731880,1196924543585
5,00258801,카카오,2024,2595101463079,491969958048,38156547286
6,00258801,카카오,2023,2499599504613,533021659728,102108806408
7,00258801,카카오,2022,2456590805969,543978112016,1611790174060
8,00258801,카카오,2021,0,397366297001,503123622467
9,00258801,카카오,2020,0,296656217228,-120417114552


In [29]:
query = "SELECT * FROM dart_report_cfs;"
check2 = pd.read_sql(query, engine)
check2

,corp_code,corp_name,year,revenue,operating_profit,net_profit
0,00266961,NAVER,2024,10737719264647,1979263438827,1931976372953
1,00266961,NAVER,2023,9670643576585,1488820269608,985017762493
2,00266961,NAVER,2022,8220078708810,1304664410543,673180300087
3,00266961,NAVER,2021,6817599707250,1325478171117,16477625559645
4,00266961,NAVER,2020,5304145900386,1215342457273,844996977149
5,00258801,카카오,2024,7871692199887,460212161525,-161870567171
6,00258801,카카오,2023,7557001757272,460857845673,-1816669011014
7,00258801,카카오,2022,6798741511168,569390445168,1067029037423
8,00258801,카카오,2021,0,587884339202,1640484195406
9,00258801,카카오,2020,0,450519574781,168978904480
